In [2]:
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
train_data = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')

In [3]:
train_doc = train_data.apply(lambda x: '%s %s' % (x['product_title'], x['product_description']), axis=1)
test_doc = test_data.apply(lambda x: '%s %s' % (x['product_title'], x['product_description']), axis=1)

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from scipy import sparse as sp

cv = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 1),
                     stop_words='english', lowercase=True)

train_count = cv.fit_transform(train_doc)
tmp_count = train_count.copy()

# calculate TF
tmp_count.data = np.log(1 + np.log(1 + tmp_count.data))

# calculate IDF
n_docs, n_words = tmp_count.shape
df = np.bincount(tmp_count.indices, minlength=tmp_count.shape[1])
idf = np.log((n_docs + 1) / df)
idf_diag = sp.spdiags(idf, diags=0, m=n_words, n=n_words)

# doc length normalization
b = 0.7
doc_lens = train_count.sum(axis=1)
doc_lens = 1 - b + b * doc_lens / doc_lens.mean()
doc_lens_diag = sp.spdiags(1/np.ravel(doc_lens), diags=0, m=n_docs, n=n_docs)


# calculate tf-idf
tfidf = doc_lens_diag * tmp_count * idf_diag


array([ 1.87997554,  0.46990503,  2.61879676, ...,  0.31871173,
        0.96460589,  2.22189113])

In [ ]:
!vim helpers.py

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, _document_frequency, bincount
from scipy import sparse as sp
tmp = np.array([[1, 0, 1], [0, 0, 0], [0, 1, 1]])

np.log(1 + np.log(tmp + 1))
TfidfVectorizer??